In [2]:
import pandas as pd
import numpy as np
import os

In [72]:
def get_diff_bin(data):
    average_bin, bin_0, bin_24 = [], [], []

    for row in range(24, data.shape[0], 25):
        bin_24.append(data.loc[row, 'bin_volume'])

    for row in range(0, data.shape[0], 25):
        bin_0.append(data.loc[row, 'bin_volume'])
        average_bin.append(data.loc[row+ 1: row+ 24, 'daily_volume'].mean())
    return average_bin, bin_0, bin_24


def transformer(data):
    transformed_nums = [-1 if num < 0 else 1 if num > 0 else 0 for num in data]
    return transformed_nums

average_bin, bin_0, bin_24 = get_diff_bin(data_46)
# print(len(average_bin), len(bin_0), len(bin_24))

diff_average_bin = transformer([(average_bin[i] - average_bin[i-1]) for i in range(1, len(average_bin))])
diff_bin_0_24 = [bin_0[i]- bin_24[i] for i in range(len(bin_24))][1:]
diff_bin_0_0 = [(bin_0[i]- bin_0[i-1])/bin_0[i-1] for i in range(1, len(bin_0))]
# print(len(diff_average_bin), len(diff_bin_0_0), len(diff_bin_0_24))

data_diff = pd.DataFrame({'ddiff_average_bin': diff_average_bin, 'bin_0_0': diff_bin_0_0, 'bin_0_24': diff_bin_0_24})
# data_diff.corr()

In [31]:
def df2matrix(file_path:str, col_name: str=None, bin_num: bool= True):
    """
    转换数据结构，以date为横坐标，制定col_name为纵坐标
    """

    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    df.set_index('date', inplace= True)
    df.sort_index(inplace= True)
    n, k = df.shape[0], 25
    # TODO: 1、把 bin0 分离出来，而后补充到特征里面去，也就是需要在  self.genNewFeatureBinVolume() 把bin0 特征融合进来
    if bin_num:
        data = pd.DataFrame(np.array(df[col_name]).reshape(int(n/k), k),
                                columns= ['bin{}'.format(i) for i in range(k)]).drop('bin0', axis=1)
    else:
        bin0_list = []
        for i in range(0, df.shape[0], 25):
            if i == 0:
                bin0_list.append([(df[col_name].iloc[i]- df[col_name].iloc[i])/ df[col_name].iloc[i]])
            else:
                bin0_list.append([(df[col_name].iloc[i]- df[col_name].iloc[i-25])/ df[col_name].iloc[i-25]])
        result = np.array([[element[0]]*(k-1) for element in bin0_list])
        print(result.shape)
        data = pd.DataFrame(np.array(result),
                            columns= ['bin0' for i in range(k-1)])
    data['date'] = pd.to_datetime(list(sorted(set(df.index))), format='%Y/%m/%d')
    data.set_index('date', inplace= True)
    data.sort_index(inplace=True)
    return data

data = df2matrix(file_path= '../data/0308/000046_XSHE_25_daily.csv', col_name= 'bin_volume', bin_num= False)

(197, 24)


In [185]:
import pandas as pd

df = pd.read_csv('../data/comment/000046.csv')
df2 = pd.read_csv('../data/0308/000046_XSHE_25_daily.csv')
# df2['date'] = pd.to_datetime(df2['date'])
# df2['date'] = df2['date'].dt.date
df['publishDate'] = pd.to_datetime(df['publishDate'])

start_time = '2020-5-12 15:00'
end_time = '2021-3-8 15:00'
filtered_df = df[(df['publishDate'] >= start_time) & (df['publishDate'] <= end_time)].copy()
filtered_df['dates'] = filtered_df['publishDate'].dt.date

In [186]:
date_XSHE = set(df2['date'])
date_XSHE

{'2020-05-13',
 '2020-05-14',
 '2020-05-15',
 '2020-05-18',
 '2020-05-19',
 '2020-05-20',
 '2020-05-21',
 '2020-05-22',
 '2020-05-25',
 '2020-05-26',
 '2020-05-27',
 '2020-05-28',
 '2020-05-29',
 '2020-06-01',
 '2020-06-02',
 '2020-06-03',
 '2020-06-04',
 '2020-06-05',
 '2020-06-08',
 '2020-06-09',
 '2020-06-10',
 '2020-06-11',
 '2020-06-12',
 '2020-06-15',
 '2020-06-16',
 '2020-06-17',
 '2020-06-18',
 '2020-06-19',
 '2020-06-22',
 '2020-06-23',
 '2020-06-24',
 '2020-06-29',
 '2020-06-30',
 '2020-07-01',
 '2020-07-02',
 '2020-07-03',
 '2020-07-06',
 '2020-07-07',
 '2020-07-08',
 '2020-07-09',
 '2020-07-10',
 '2020-07-13',
 '2020-07-14',
 '2020-07-15',
 '2020-07-16',
 '2020-07-17',
 '2020-07-20',
 '2020-07-22',
 '2020-07-23',
 '2020-07-24',
 '2020-07-27',
 '2020-07-28',
 '2020-07-29',
 '2020-07-30',
 '2020-07-31',
 '2020-08-03',
 '2020-08-04',
 '2020-08-05',
 '2020-08-06',
 '2020-08-07',
 '2020-08-10',
 '2020-08-11',
 '2020-08-12',
 '2020-08-13',
 '2020-08-14',
 '2020-08-17',
 '2020-08-

In [208]:
def calculate_time_bins(day):
    bins = {}
    # bin_0: 从前一天的15:00到当天的09:30
    bin_0_start = f'{day - pd.Timedelta(days=1)} 15:00'
    bin_0_end = f'{day} 09:30'
    bins['bin0'] = [bin_0_start, bin_0_end]

    # 初始化bin_1的起始时间为bin_0的结束时间
    current_start = bin_0_end

    # 循环生成剩下的23个bin
    for i in range(1, 25):
        if i == 10:
            bins[f'bin{i}'] = [current_start, f'{day} 11:30']
            current_start = f'{day} 11:30'
        elif i == 11:
            bins[f'bin{i}'] = [f'{day} 11:30', f'{day} 13:00']
            current_start = f'{day} 13:00'
        elif i == 24:
            bins[f'bin{i}'] = [current_start, f'{day} 15:00']
        else:
            current_end = pd.to_datetime(current_start) + pd.Timedelta(minutes=9.875)
            bins[f'bin{i}'] = [current_start, current_end.strftime('%Y-%m-%d %H:%M')]
            current_start = current_end.strftime('%Y-%m-%d %H:%M')
    return bins

times_set = set()
for i in filtered_df['dates']:
    times_set.add(i)

data_bin = pd.DataFrame(index = list(times_set),
                        columns= [f'bin{i}' for i in range(0, 25)])
data_bin.index = pd.to_datetime(data_bin.index)
data_bin = data_bin.sort_index(ascending= True)
for day in data_bin.index[1:]:
    # print()
    if str(day.date()) in date_XSHE:
        bin_newstitle = {}
        bins = calculate_time_bins(day.date())
        for bin in bins:
            bin_newstitle[bin] = filtered_df[(filtered_df['publishDate'] >= bins[bin][0]) & (filtered_df['publishDate'] <= bins[bin][1])].loc[:, 'newsTitle'].values
        data_bin.loc[day] = bin_newstitle

In [214]:
print(data_bin.iloc[10,6], data_bin.index[10])

['老卢都跑了，韭菜们相互扶持吧。' '操盘手感觉到没意思了，只能一手一手向下砸，没人理他。[大笑]'] 2020-05-22 00:00:00


In [156]:
bins

{'bin0': ['2021-03-07 15:00', '2021-03-08 09:30'],
 'bin1': ['2021-03-08 09:30', '2021-03-08 09:39'],
 'bin2': ['2021-03-08 09:39', '2021-03-08 09:48'],
 'bin3': ['2021-03-08 09:48', '2021-03-08 09:57'],
 'bin4': ['2021-03-08 09:57', '2021-03-08 10:06'],
 'bin5': ['2021-03-08 10:06', '2021-03-08 10:15'],
 'bin6': ['2021-03-08 10:15', '2021-03-08 10:24'],
 'bin7': ['2021-03-08 10:24', '2021-03-08 10:33'],
 'bin8': ['2021-03-08 10:33', '2021-03-08 10:42'],
 'bin9': ['2021-03-08 10:42', '2021-03-08 10:51'],
 'bin10': ['2021-03-08 10:51', '2021-03-08 11:30'],
 'bin11': ['2021-03-08 11:30', '2021-03-08 13:00'],
 'bin12': ['2021-03-08 13:00', '2021-03-08 13:09'],
 'bin13': ['2021-03-08 13:09', '2021-03-08 13:18'],
 'bin14': ['2021-03-08 13:18', '2021-03-08 13:27'],
 'bin15': ['2021-03-08 13:27', '2021-03-08 13:36'],
 'bin16': ['2021-03-08 13:36', '2021-03-08 13:45'],
 'bin17': ['2021-03-08 13:45', '2021-03-08 13:54'],
 'bin18': ['2021-03-08 13:54', '2021-03-08 14:03'],
 'bin19': ['2021-03-08